In [42]:
import random
import torch
from torch.distributions.multinomial import Multinomial

In [43]:
num_tosses = 1000
heads = sum([random.random() > 0.5 for _ in range(num_tosses)])
tails = num_tosses - heads
print("heads, tails: ", [heads, tails])

heads, tails:  [518, 482]


In [44]:
fair_probs = torch.tensor([0.5, 0.5])
Multinomial(100, fair_probs).sample()

tensor([47., 53.])

In [45]:
Multinomial(100, fair_probs).sample() / 100

tensor([0.4000, 0.6000])

In [46]:
counts = Multinomial(10000, fair_probs).sample()
counts / 10000

tensor([0.5032, 0.4968])



## 1. 贝叶斯公式的推导

* 条件概率的定义：

  $$
  P(A,B) = P(A|B)P(B) = P(B|A)P(A)
  $$
* 两边相等，得到：

  $$
  P(A|B) = \frac{P(B|A)P(A)}{P(B)}.
  $$
* 这就是经典的 **贝叶斯公式**。

---

## 2. 贝叶斯公式的直观意义

* 它可以 **反转条件概率的方向**。
  通常我们容易得到 $P(B|A)$（例如：已知疾病 $A$ 时出现症状 $B$ 的概率），但我们真正想要的是 $P(A|B)$（出现症状后有病的概率）。
* 贝叶斯公式帮我们通过 $P(B|A)$、先验 $P(A)$、以及边缘概率 $P(B)$ 计算出需要的 $P(A|B)$。

---

## 3. 简化版公式

有时候我们不知道 $P(B)$，但可以用比例的方式写：

$$
P(A|B) \propto P(B|A)P(A).
$$

* 意思是：后验概率和“似然 × 先验”成正比。
* 最后还要通过归一化确保所有可能的 $A$ 概率和为 1。

---

## 4. 归一化（正规化）

为了把“成比例”变成“等于”，需要归一化：

$$
P(A|B) = \frac{P(B|A)P(A)}{\sum_a P(B|A=a)P(A=a)}.
$$

* 分母就是所有可能 $A$ 的情况求和，保证结果是一个真正的概率分布。

---

## 5. 在贝叶斯统计里的解释

* **Prior 先验** $P(H)$：在看到数据前，对假设 $H$ 的主观信念。
* **Likelihood 似然** $P(E|H)$：如果假设 $H$ 成立，看到证据 $E$ 的可能性。
* **Posterior 后验** $P(H|E)$：看到数据后，更新对假设的信念。
* 公式：

  $$
  P(H|E) = \frac{P(E|H)P(H)}{P(E)}.
  $$

  可以理解为：**后验 = 先验 × 似然 ÷ 证据**。

---

## 6. 边缘化 (Marginalization)

分母 $P(E)$ 实际上是把所有可能的假设都考虑进来：

$$
P(E) = \sum_H P(E|H)P(H).
$$

这个求和过程叫做 **边缘化**，本质就是把联合概率 $P(E,H)$ 对 $H$ 求和，得到边缘分布 $P(E)$。

---

## 7. 小结

* 贝叶斯公式 = 条件概率公式的直接推论。
* 它的价值在于**反转条件概率方向**，把 $P(B|A)$ 变成 $P(A|B)$。
* 在贝叶斯统计里：

  * **先验**：主观信念。
  * **似然**：假设解释数据的能力。
  * **后验**：结合数据更新后的信念。
* 分母 $P(B)$ 通过边缘化得到，确保归一化。

---



---

## 1. 什么是期望 (Expectation)

* 期望就是“平均结果”。
* 在随机变量 $X$ 上，它表示 **长期重复实验时的平均值**。
* 定义（离散型）：

  $$
  E[X] = \sum_x x \, P(X=x).
  $$
* 定义（连续型）：

  $$
  E[X] = \int x \, p(x)\, dx.
  $$

例子：投资可能 50% 完全失败 (收益 0)，40% 有 2 倍收益，10% 有 10 倍收益：

$$
E[X] = 0.5\cdot 0 + 0.4\cdot 2 + 0.1\cdot 10 = 1.8.
$$

所以期望回报是 1.8 倍。

---

## 2. 函数的期望

不仅可以算随机变量本身的期望，还可以算它的函数 $f(X)$ 的期望：

$$
E[f(X)] = \sum_x f(x)P(x)\quad\text{或}\quad \int f(x)p(x)\, dx.
$$

在经济学里，这个 $f$ 常常表示 **效用函数 (utility)**，即“幸福感”。

* 人们对钱的效用往往是**非线性**的（损失更痛苦，收益边际递减）。
* 例如“金钱效用是对数函数”的说法，就是这个原因。

---

## 3. 投资效用的例子

如果设定：

* 完全失败：效用 = -1
* 收益 1 倍 → 效用 1
* 收益 2 倍 → 效用 2
* 收益 10 倍 → 效用 4

那么期望效用：

$$
E[f(X)] = 0.5\cdot (-1) + 0.4\cdot 2 + 0.1\cdot 4 = 0.7.
$$

代表预期幸福值是 **负的**（比不投资更糟），所以理性的选择可能是不要投资。

---

## 4. 风险与方差 (Variance)

除了期望，还要考虑 **风险性**，即结果波动有多大。

* 定义：

  $$
  \text{Var}[X] = E\left[(X-E[X])^2\right] = E[X^2] - (E[X])^2.
  $$
* 标准差 $\sigma = \sqrt{\text{Var}[X]}$ 具有和 $X$ 相同的单位，更直观。

例子：投资例子中的方差：

$$
\text{Var}[X] = 0.5\cdot 0^2 + 0.4\cdot 2^2 + 0.1\cdot 10^2 - 1.8^2 = 8.36.
$$

说明投资虽然期望收益高，但波动性极大 → 风险高。

---

## 5. 多元情况：均值向量与协方差矩阵

* 对向量随机变量 $\mathbf{x}$，期望就是分量逐一的平均：

  $$
  \mu = E[\mathbf{x}] \quad (\mu_i = E[x_i]).
  $$
* 协方差矩阵：

  $$
  \Sigma = E\big[(\mathbf{x}-\mu)(\mathbf{x}-\mu)^T\big].
  $$
* 性质：对任意向量 $\mathbf{v}$，

  $$
  \mathbf{v}^T \Sigma \mathbf{v} = \text{Var}(\mathbf{v}^T\mathbf{x}).
  $$

  这表示我们能通过 $\Sigma$ 计算任何线性组合的方差。
* 协方差矩阵的对角线是方差，非对角线表示变量之间的相关性（0=无关，正/负值表示正/负相关）。

---

## 6. 总结

这一节的主线：

1. **期望**：平均值，衡量“长期收益”。
2. **效用期望**：考虑心理价值，效用函数可非线性。
3. **方差**：衡量不确定性/风险。
4. **多元扩展**：期望向量、协方差矩阵，用来刻画多个变量的均值和相关性。



# Exercises


### 1. 什么时候更多数据能把不确定性降到任意低？

**例子**：掷一枚可能存在偏差的硬币。

* 我们不知道硬币正面朝上的真实概率 $p$。
* 如果我们不断收集掷硬币的数据（例如上万次投掷），我们就可以把对 $p$ 的估计精确到非常小的误差。
* 这时的不确定性来自 **参数未知**，属于 **epistemic uncertainty**。随着数据量无限增大，我们对参数的估计可以无限逼近真实值。

➡️ 解释：这是 **模型参数的不确定性**，可以通过数据收集逐渐消除。

---

### 2. 什么时候更多数据只能降低到一定程度？

**例子**：预测下一次公平硬币投掷的结果。

* 即使我们完全知道概率 $p=0.5$，下一次投掷结果仍然不可预测，只能说有 50% 的概率是正面。
* 这种不确定性来自事件本身的 **随机性 (aleatoric uncertainty)**，数据再多也无法消除。

➡️ 解释：这种不确定性是问题固有的噪声，无法通过收集更多数据进一步减少。

* 在这个例子里，收集数据最多只能让我们确定“硬币是公平的”，之后不确定性停留在 $p=0.5$ 上，再也无法减少。

---

## 总结：

1. **更多数据可无限降低的不确定性** → 参数未知 (epistemic)。例：硬币偏向概率估计。
2. **更多数据无法完全消除的不确定性** → 问题固有随机性 (aleatoric)。例：下一次硬币投掷结果。

---



设 $X_i\sim \text{Bernoulli}(p)$ 表示第 $i$ 次掷硬币是否为正面（1 为正面，0 为反面），独立同分布。
用样本频率

$$
\hat p=\frac1n\sum_{i=1}^n X_i
$$

估计正面概率 $p$。

## (1) 方差随样本数的缩放

$$
\mathbb E[\hat p]=p,\qquad 
\operatorname{Var}(\hat p)=\operatorname{Var}\!\left(\frac1n\sum X_i\right)
=\frac{1}{n^2}\sum \operatorname{Var}(X_i)
=\frac{n\cdot p(1-p)}{n^2}=\frac{p(1-p)}{n}.
$$

→ 方差 **$O(1/n)$**；样本数翻 10 倍，方差缩至 1/10。最坏情况下 $p(1-p)\le 1/4$。

## (2) 用切比雪夫不等式给出偏离期望的上界

对任意 $\varepsilon>0$：

$$
\Pr\!\left(|\hat p-p|\ge \varepsilon\right)
\le \frac{\operatorname{Var}(\hat p)}{\varepsilon^2}
= \frac{p(1-p)}{n\varepsilon^2}
\le \frac{1}{4n\varepsilon^2}.
$$

这是分布无关（只用到了方差）的保守上界。

## (3) 与中心极限定理（CLT）的关系

CLT 给出更精细的近似：

$$
\sqrt{n}\,\frac{\hat p-p}{\sqrt{p(1-p)}} \ \xrightarrow{d}\ \mathcal N(0,1),
$$

等价地，

$$
\hat p \approx \mathcal N\!\left(p,\ \frac{p(1-p)}{n}\right)\quad (n\ \text{大}).
$$

因此可得近似置信区间

$$
\hat p \pm z_{\alpha/2}\sqrt{\frac{p(1-p)}{n}}
$$

（实际常用 $\hat p$ 代替 $p$ 做 plug-in）。
和切比雪夫相比，CLT 在样本足够大时给出**更紧**的概率界与区间（例如 95% 时用 $z_{0.025}\approx1.96$，而切比雪夫要 $k\approx 4.47$ 才到 95%）。


In [47]:
## TODO exercises 5-8